<a href="https://colab.research.google.com/github/seungbinahn/START_AI/blob/master/02_Machine_Learning%E2%80%8E/07_1_MNIST_%EC%9D%B8%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST

* 딥러닝 모델의 'Hello world' 데이터 셋
* 0~9 사이의 숫자를 손글씨로 쓴 이미지 데이터 셋
* 입력 데이터 : 28*28 픽셀에 해당하는 0~255사이의 값
* 타겟 데이터(label) : 0~9 사이의 값

## 구글 드라이브와 연동

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!pwd # 리눅스 명령어는 ! 사용

/content


In [ ]:
data_path = '/content/gdrive/My Drive/Colab Notebooks/data/sample_weight.pkl'

In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np

In [ ]:
(x_train, y_train), (x_test,y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


## 데이터 전처리
* flatten : 이미지 배열(28*28) 이 60000개 존재 : 이를 쭉 펴줌
* normalization : 0 ~ 255 범위인 각 픽셀의 값을 0.0 ~ 1.0 범위로 변환

In [ ]:
x_train = x_train.reshape(60000, 28*28) # flatten
x_train = x_train.astype('float32') / 255 # normalization

x_test = x_test.reshape(10000, 28*28) # flatten
x_test = x_test.astype('float32') / 255 # normalization

## 활성화 함수 구현

In [ ]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(a):
  """
  값 리스트를 확률 분포로 변화하는 함수

  a : 값 리스트
  """
  exp_a = np.exp(a) # 지수 함수 적용
  sum_exp_a = np.sum(exp_a) # 지수 함수의 합
  y = exp_a / sum_exp_a # 이산 확률분포로 변환
  
  return y

## 예측함수 구현

In [ ]:
import pickle

def init_network():
    """
    pickle 파일로부터 모델 로드
    """
    with open(data_path, 'rb') as f:
        network = pickle.load(f)

    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

## 결과

In [ ]:
network = init_network()

accuracy_cnt = 0
for i in range(len(x_train)):
    y = predict(network,x_train[i])
    p = np.argmax(y) # 확률이 가장 높은 원소의 인덱스
    if p == y_train[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x_train)))

Accuracy:0.9357666666666666


# Batch 처리
* 한장씩 처리하는것이 아니라 동시에 여러장 실행

In [ ]:
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x_train), batch_size): # batch 단위로 잘라서 실행
    x_batch = x_train[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis = 1) # 1번째 차원을 축으로 최대값 인덱스 탐색
    accuracy_cnt += np.sum(p == y_train[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x_train)))

Accuracy:0.9357666666666666


## np.argmax 함수
리스트에서 최대값의 인덱스를 리턴함

In [ ]:
x = np.array([[1,2,3,4],[4,5,4,5],[9,8,7,6],[2,2,2,1]])
y = np.argmax(x, axis = 1)
y

array([3, 1, 0, 0])